This one heavily inspired by https://www.kaggle.com/cbrogan/titanic/xgboost-example-python/files

In [15]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import Imputer,LabelEncoder
from sklearn.base import TransformerMixin

In [10]:
# taken from http://stackoverflow.com/questions/25239958/impute-categorical-missing-values-in-scikit-learn
#updated for ints

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() if X[c].dtype == np.dtype('float64') else X[c].median() 
                               for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)


In [11]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [13]:
keysToUse = set(train.keys())

keysToUse.remove('ID')
keysToUse.remove('target')
keysToUse = list(keysToUse)

nonNumericKeys = list(train.select_dtypes(include=['object']).keys())

In [14]:
bigX = train[keysToUse].append(test[keysToUse])
bigXimputed = DataFrameImputer().fit_transform(bigX)

In [16]:
le = LabelEncoder()
for col in nonNumericKeys:
    bigXimputed[col] = le.fit_transform(bigXimputed[col])

In [22]:
trainX = bigXimputed[:train.shape[0]].as_matrix()
testX = bigXimputed[train.shape[0]:].as_matrix()
trainY = train['target']

In [33]:
gbm = xgb.XGBClassifier(max_depth=8, n_estimators=300, learning_rate=0.04,nthread=4,subsample=0.9,
                        colsample_bytree=0.9)
gbm.fit(trainX, trainY)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.9,
       gamma=0, learning_rate=0.04, max_delta_step=0, max_depth=8,
       min_child_weight=1, missing=None, n_estimators=300, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.9)

In [34]:
predictions = gbm.predict_proba(testX)[:,1]
print predictions

[ 0.29482785  0.90736604  0.88187772 ...,  0.87648571  0.93920881
  0.48620355]


In [35]:
output = pd.DataFrame({'ID': test['ID'],'PredictedProb': predictions})
output.to_csv('submission.csv',index=False)

In [32]:
?xgb.XGBClassifier